In [45]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
import seaborn as sn
import pandas as pd
from scipy import signal
from scipy import stats
import ipywidgets as widgets
from scipy.fftpack import fft
from scipy.fftpack import ifft
from scipy.signal import welch
from detecta import detect_peaks
import time
import datetime
import random
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import PredefinedSplit
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix
# from xgboost import XGBClassifier
import dill
import pywt
from collections import Counter
from collections import defaultdict
import pickle
import lime
import lime.lime_tabular
# import pdfkit
import warnings
warnings.simplefilter('ignore')
from platform import python_version
print(python_version())

3.9.2


# Data Preparation

In [2]:
def readData(accDir, annotFile):
    files = os.listdir(accDir)
    files_csv = [f for f in files if f[-3:] == 'csv']
    empatica_dict = dict()
    for f in files_csv:
        data = np.genfromtxt(accDir+f, delimiter=',') # creates numpy array for each Empatica acc csv file
        key = int(float(f.strip("ACC.csv")))
        empatica_dict[key] = data
    tmp = pd.read_excel(annotFile, sheet_name=None, usecols='A:B')
    annot_dict = dict(zip(tmp.keys(), [i.dropna() for i in tmp.values()])) # Remove the rows with NaN values (some with ladder 2 missing)
    return empatica_dict, annot_dict

def getLabeledDict(empatica_dict, annot_dict, subject_ids):
    labeled_dict = {}; taskInd_dict = {}
    for id in subject_ids:
        start_time = int(empatica_dict[id][0,0])
        acc = empatica_dict[id][2:,:]
        label = list(map(lambda i: i.replace("_end", "").replace("_start", ""), annot_dict['P'+ str(id)].taskName.tolist()))
        task_time= list(map(lambda i: time.mktime(datetime.datetime.strptime(i[:6] + '20' + i[6:], "%m/%d/%Y %H:%M:%S").timetuple()),
                            annot_dict['P'+ str(id)].startTime_global.tolist()))
        task_ind = [int(x - start_time)*SR for x in task_time]
        taskInd_dict[id] = task_ind
        label_tmp = np.empty(acc.shape[0], dtype=object)
        for i, (j, k) in enumerate(zip(task_ind[0::2], task_ind[1::2])):
            tmpInd = 2*i
            label_tmp[j:k] = label[tmpInd]
        labeled_dict[id] = np.hstack((acc, label_tmp.reshape(label_tmp.shape[0],1)))
    return labeled_dict, taskInd_dict

In [3]:
sepAccDict, sepAnnotDict = readData(accDir='Data/Acc Data/separate/', annotFile='Data/Annotation Data/separate.xlsx')
SR=int(sepAccDict[8][1,0])

sepSubIDs = list(range(8,45))
sepLabeledDict, sepTaskIndDict = getLabeledDict(sepAccDict, sepAnnotDict, sepSubIDs)

In [4]:
def getArrDict(labeledDict, subject_ids, windowLen):
    ArrDict = {}
    for id in subject_ids:
        df = pd.DataFrame(data=labeledDict[id], columns=['aX', 'aY', 'aZ', 'label'])
        grDict = dict(tuple(df.groupby(by='label'))) # dict values are pd DF's
        sigArr = np.zeros((0, windowLen, 4))
        for key in grDict.keys():
            task_df = grDict[key]
            tmp1 = [task_df.iloc[i:i+windowLen, :] for i in range(0, task_df.shape[0], windowLen)]
            if len(tmp1[-1]) != windowLen: del tmp1[-1]
            sigArr = np.vstack((sigArr, np.array(tmp1)))
        ArrDict[id] = sigArr
    return ArrDict

In [5]:
def applyFilter(sig, SR, n, fc):
    w = fc/(SR/2)
    b, a = signal.butter(n, w, 'low')
    filteredSig = signal.filtfilt(b, a, sig)
    return filteredSig

def fftCalc(sig, sampRate, windowLen):
    f_values = np.linspace(0, sampRate/2, windowLen//2)
    fft_values = 2/windowLen * np.abs(fft(sig)[0:windowLen//2])
    filtered_f = f_values[f_values<10]
    filtered_fft = fft_values[f_values<10]
    return filtered_f, filtered_fft

def fftPeaks(sig, filtered_f, filtered_fft, percentile, denominator, sampRate, no_peaks):
    signal_min = np.percentile(sig, percentile)
    signal_max = np.percentile(sig, 100-percentile)
    mph = (signal_max - signal_min)/denominator # minimum peak height
    indices_peaks = detect_peaks(filtered_fft, mph=mph)
    if len(indices_peaks) >= no_peaks:
        peakFeatures = list(filtered_f[indices_peaks])[:no_peaks] + list(filtered_fft[indices_peaks])[:no_peaks]
    else:
        missing = no_peaks-len(indices_peaks)
        peakFeatures = list(filtered_f[indices_peaks])+[0]*missing + list(filtered_fft[indices_peaks])+[0]*missing
    return peakFeatures

def getStats(sig):
    mean_ = np.mean(sig)
    rms_ = np.sqrt(np.mean(np.square(sig)))
    mad_ = np.mean(np.absolute(sig-np.mean(sig)))
    std_ = np.std(sig)
    min_ = min(sig)
    max_ = max(sig)
    med_ = np.percentile(sig, 50)
    perc25 = np.percentile(sig, 25)
    perc75 = np.percentile(sig, 75)
    stats_ = [mean_, rms_, mad_, std_, min_, max_, med_, perc25, perc75]
    return stats_

def getCrossings(sig):
    zeroCross = np.count_nonzero(np.diff(np.sign(sig)))
    centeredSig = sig - np.mean(sig)
    meanCross = np.count_nonzero(np.diff(np.sign(centeredSig)))
    return [zeroCross, meanCross]

def getEntropy(sig):
    count = Counter(sig).most_common()
    probability = [elem[1]/len(sig) for elem in count]
    return [stats.entropy(probability)]

def extract_features_dicts(dataset, key, windowLen, sampRate, denominator, no_peaks, waveletName, featureSet, percentile=5):
    list_of_features = []
    for samp_no in range(0, dataset.shape[0]):
        features = []
        for sig_comp in range(0, dataset.shape[2] - 1):
            signal = dataset[samp_no, :, sig_comp]
            if featureSet == 'Time':
                features += getStats(applyFilter(signal, SR=sampRate, n=4, fc=10))
            elif featureSet == 'FrequencyAll':
                filtered_f, filtered_fft = fftCalc(signal, sampRate, windowLen)
                features += list(filtered_fft)
            elif featureSet == 'FrequencyPeaks':
                filtered_f, filtered_fft = fftCalc(signal, sampRate, windowLen)
                features += fftPeaks(signal, filtered_f, filtered_fft, percentile, denominator, sampRate, no_peaks)
            elif featureSet == 'TimeFrequency':
                waveletCoeffs = pywt.wavedec(signal, waveletName)
                for coeff in waveletCoeffs:
                    features += getStats(coeff) + getEntropy(coeff) + getCrossings(coeff)
            
        modifiedLabel = ''.join([i for i in dataset[samp_no, 0, 3] if not i.isdigit()]) # remove any numbers from label
        features.append(modifiedLabel) # Add modified label as last column
        features.append(key)
        list_of_features.append(features)
    features_df = pd.DataFrame(np.array(list_of_features), columns=list(range(1, len(list_of_features[0]) -1)) + ['label', 'subID'])
    return features_df

# Train and Test

In [6]:
def GetTrainTestData(featureDict, trainSub, testSub):
    test_df = pd.concat([featureDict[x] for x in testSub]).reset_index().drop(['index'], axis=1)
    X_test = test_df.loc[:, test_df.columns != 'label']
    Y_test = test_df.label
    trainDict = {}
    for key in trainSub:
        trainDict[key] = featureDict[key]
        trainDict[key]['subID'] = key
    train_df = pd.concat([trainDict[x] for x in trainDict.keys()]).reset_index().drop(['index'], axis=1)
    X_train = train_df.drop(['label', 'subID'], axis=1)
    Y_train = train_df.label
    return train_df, X_train, Y_train, X_test, Y_test

In [7]:
def GetTestTrainSubjects(sub_list, fold, num_of_folds):
    random.seed(1365)
    random.shuffle(sub_list)
    test_subs = sub_list[fold-1::num_of_folds]
    train_subs = list(set(sub_list) - set(test_subs))
    return test_subs, train_subs

# Get Time-Frequency Features

In [12]:
windowLen = 320 # 320 or 128
sepArrDict = getArrDict(sepLabeledDict, sepSubIDs, windowLen)
feature = 'TimeFrequency'# 'FrequencyAll' or 'TimeFrequency'

sub_list = list(sepArrDict.keys())
test_subs, train_subs = GetTestTrainSubjects(sub_list, fold=1, num_of_folds=5)

features_df = {}
for key in sepArrDict.keys():
    features_df[key] = extract_features_dicts(dataset=sepArrDict[key], key=key, windowLen=windowLen, sampRate=SR, denominator=15,
                                              no_peaks=5, waveletName='db4', featureSet=feature, percentile=5)
train_df, X_train_df, Y_train, X_test_df, Y_test = GetTrainTestData(features_df, train_subs, test_subs)
X_train = StandardScaler().fit_transform(X_train_df)
X_test = StandardScaler().fit_transform(X_test_df.iloc[:,:-1])

## Import Fold 1,  *k*-NN, 10 Seconds windows, Frequency and Time-Frequency Features

In [15]:
infile = open('./outputs/classification/inter-subject/Fold1_FrequencyAll_KNN_320.pickle', 'rb')
Frequency_results = pickle.load(infile)
infile.close()
clf_best_Freq = Frequency_results['clf_best']
Y_pred_Freq = Frequency_results['TruePredY'].Predicted

infile = open('./outputs/classification/inter-subject/Fold1_TimeFrequency_KNN_320.pickle', 'rb')
Time_Frequency_results = pickle.load(infile)
infile.close()
clf_best_Time_Freq = Time_Frequency_results['clf_best']
Y_pred_Time_Freq = Time_Frequency_results['TruePredY'].Predicted

### Set Names for Features (Time-Freq)

In [16]:
featureNames = []
for i in ['X', 'Y', 'Z']:
    for j in ['Approx-5', 'Deatil-5', 'Detail-4', 'Detail-3', 'Detail-2', 'Detail-1']:
        for k in ['Mean', 'RMS', 'MAD', 'STD', 'Min', 'Max', 'Median', '25-Perc', '75-Perc', 'Entropy',
                  '0-Cross', 'Mean-Cross']:
            featureNames.append(k + ' ' + j + ' ' + i)

X_test_modified = X_test_df.copy()
X_test_modified['label'] = Y_test
X_test_modified['Freq_predicted'] = Y_pred_Freq
X_test_modified['Time_Freq_predicted'] = Y_pred_Time_Freq

## LIME for all Activities

> **Note:** The following code snippet takes around `7 hours` to run. Its output is saved in "outputs/lime/Wavelet_allActivities" directory and the following code snippets read the files from there to create the plots.

In [40]:
# start_time = time.time()
# for label_ in X_test_modified.label.unique():
#     Idx_for_exp = np.where(X_test_modified.label.unique()==label_)[0][0]
#     Idxs = X_test_modified[(X_test_modified.label==label_) & (X_test_modified.Freq_predicted==label_) & (X_test_modified.Time_Freq_predicted==label_)].index.values
#     random.seed(2021)
#     selected_obseravtions_panel = random.sample(set(Idxs), 30)
#     selected_features_proba_panel = []
#     for samp_Idx in selected_obseravtions_panel:
#         observation_features = X_test[samp_Idx,:]
#         explainer = lime.lime_tabular.LimeTabularExplainer(X_train, feature_names=featureNames, class_names=np.unique(train_df.label),
#                                                            discretize_continuous=True, random_state=1365)
#         exp = explainer.explain_instance(observation_features, clf_best_Time_Freq.predict_proba, num_features=30 , top_labels=1)
#         selected_features_proba_panel.extend(exp.as_map()[Idx_for_exp])
        
#     with open('./outputs/lime/Wavelet_allActivities/LIME_Wavelet_' + label_ + '.pickle', 'wb') as outfile:
#         pickle.dump(dict(selected_features_proba_panel=selected_features_proba_panel,
#                          selected_obseravtions_panel=selected_obseravtions_panel), outfile)

# print('Elapsed Time = {:0.0f} seconds'.format(time.time() - start_time))

Elapsed Time = 22774 seconds


### Create Probability Bar Graphs for the 30 Most Frequently Selected Features

In [42]:
lime_dict = {}
for label_ in X_test_modified.label.unique():
    with open('./outputs/lime/Wavelet_allActivities/LIME_Wavelet_' + label_ + '.pickle', 'rb') as infile:
        pickle_dict_panel = pickle.load(infile)

    selected_features_proba_panel = pickle_dict_panel['selected_features_proba_panel']
    selected_obseravtions_panel = pickle_dict_panel['selected_obseravtions_panel']
    
    ############# Find indices of the 30 most frequently selected features
    features_ = [elem[0] for elem in selected_features_proba_panel]
    most_frequent_ = Counter(features_).most_common(30)
    LIME_selected_feature_Idxs = [elem[0] for elem in most_frequent_]

    ############# Create dictionary with unique features as key and the list of its probabilities
    res = defaultdict(list)
    for i,j in selected_features_proba_panel:
        res[i].append(j)
    feature_proba_dict_ = dict(res)

    ############ Use indices of most frequent features to select
    probabilities_ = [feature_proba_dict_[key] for key in LIME_selected_feature_Idxs]
    feature_proba_dict = dict(zip(LIME_selected_feature_Idxs, probabilities_))

    ############# Create dictionary with feature name as key and mean of probabilities as values; then sort based on dict absolute values
    mean_probabilities = list(map(lambda x: np.mean(x), list(feature_proba_dict.values())))
    feature_meanProba_dict = dict(zip(np.array(featureNames)[LIME_selected_feature_Idxs], mean_probabilities))
    sorted_list_ofTuples = sorted(feature_meanProba_dict.items(), key=lambda i: abs(i[1]))
    sorted_dict = dict(sorted_list_ofTuples)
    
    lime_dict[label_] = sorted_dict

# Visualize LIME Output for all Activities in Tabs

> **Note:** The plots from the following codesnippet are created in tabsets and thus not visible in the notebook. You can see them in the output html under "outputs/htmls".

In [43]:
rc('text', usetex=True)
rc('font', weight='bold')
rcParams['text.latex.preamble'] = [r'\usepackage{sfmath} \boldmath']

In [46]:
labels_tabs = list(lime_dict.keys())
sub_tab=[widgets.Output() for i in range(len(labels_tabs))]
tab = widgets.Tab(sub_tab)
for i, act in enumerate(labels_tabs):
    tab.set_title(i, act)
    sorted_dict = lime_dict[act]
    with sub_tab[i]:
        y_pos = np.arange(len(sorted_dict))
        f, ax = plt.subplots(1,figsize=(19,10))
        ax.barh(y_pos, sorted_dict.values(), color='maroon')
        ax.set_yticks(y_pos)
        ax.set_yticklabels(sorted_dict.keys(), fontweight='bold')
        ax.set_xlim([-0.05, 0.205])
        ax.set_ylim([-1, 32])
        ax.text(0.025, 30.5, r'\textbf{' + act + '}', fontsize=20, c='mediumblue')
        ax.text(-0.04, 30.5, r'\textbf{Not ' + act + '}', fontsize=20, c='mediumblue')
        ax.set_title(r'\textbf{Probability of the 30 Most Frequently Selected Features}', fontsize=20)
        ax.set_xlabel(r'\textbf{Probability}', fontsize=20)
        ax.set_ylabel(r'\textbf{Most Frequently Selectd Features}', fontsize=20)
        ax.tick_params(labelsize=15)
        f.tight_layout()
        f.savefig('./outputs/lime/Wavelet_allActivities/' + act + '_Wavelet_proba.png')
        plt.show()

display(tab)